<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/numba-cuda/20_probability/25_regression_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Example : Linear Regression using PyTorch<br>사례 : PyTorch를 이용한 선형회귀

PyTorch is a library specialized in (computational) machine learning originally developed by Meta AI<br>PyTorch 는 Meta AI에서 기원한 (전산) 기계 학습에 특화된 라이브러리


In [ ]:
import os

import torch
import torch.nn



* Prepare data<br>데이터를 준비



In [ ]:
import numpy as np
import numpy.random as nr


nr.seed()


a = 0.5
b = 2.0

x_array = np.linspace(0, 5, 100 + 1)
y_true = a * x_array + b

w_array = nr.normal(0, 0.25, size=x_array.shape)
y_measurement = y_true + w_array



* Let's plot the data<br>데이터를 한번 그려보자



In [ ]:
import matplotlib.pyplot as plt


plt.plot(x_array, y_true, label='true')
plt.plot(x_array, y_measurement, '.', label='measurement')
plt.legend(loc=0)
plt.grid(True)



* Declare the linear model<br>선형 모델을 선언



In [ ]:
class LinearRegression(torch.nn.Module):
  def __init__(self, in_size=1, out_size=1):
    super(LinearRegression, self).__init__()
    self.linear = torch.nn.Linear(in_size, out_size)

  def forward(self, x):
    return self.linear(x)



* Instantiate the linear model<br>`LinearRegression` 클래스의 객체를 만듦 (객체는 메모리를 차지할 것임)



In [ ]:
custom_model = LinearRegression()



* It randomly initializes the weight and bias.<br>가중치와 편향은 무작위로 초기화됨



In [ ]:
w, b = list(custom_model.parameters())
w.item(), b.item()



* Convert `numpy.array` to `float64` `torch.Tensor`<br>`numpy.array` 를 `torch.Tensor`로 변환 (각 원소는 `float64`)



In [ ]:
x_tensor_float64 = torch.from_numpy(x_array)
y_tensor_float64 = torch.from_numpy(y_measurement)

x_tensor_float64



* Convert `float64` to `float32`<br>`float64` 를 `float32`로 변환



In [ ]:
x_tensor_float32 = x_tensor_float64.float()
y_tensor_float32 = y_tensor_float64.float()
x_tensor_float32



* Convert to $n \times 1$ tensor to match model weight dimension<br>모델 가중치의 차원과 맞추기 위해 $n \times 1$ 텐서로 변환



In [ ]:
x_tensor = x_tensor_float32.view(-1, 1)
y_tensor = y_tensor_float32.view(-1, 1)
x_tensor.shape



* Make prediction (using random weight and bias)<br>(무작위 기울기와 절편으로) 예측을 시도해 보자.



In [ ]:
y_hat_tensor = custom_model(x_tensor)



In [ ]:
def plot(x_array, y_true, y_measurement, y_hat_tensor, y_hat_label):
  plt.plot(x_array, y_true, label='true')
  plt.plot(x_array, y_measurement, '.', label='measurement')
  plt.plot(
      x_array, y_hat_tensor.detach().numpy(), '.',
      label=y_hat_label
  )
  plt.legend(loc=0)
  plt.grid(True)



In [ ]:
plot(x_array, y_true, y_measurement, y_hat_tensor, 'initial prediction')



* Let's use (Stochastic) Gradient Descent for the optimizer<br>최적화 방안으로 (확률적) 경사 하강법을 선택 해 보자.



In [ ]:
optimizer = torch.optim.SGD(custom_model.parameters(), lr=0.1)



* Mean Square Error will be our loss function.<br>손실 함수로 평균 제곱 오차 (MSE) 를 사용하자.



In [ ]:
criterion = torch.nn.MSELoss()



* Let's train the model<br>모델을 학습시켜 보자.



In [ ]:
def train(y_tensor, x_tensor, model, optimizer, criterion, n_epoch=1000):
  cost = []
  w_list = []
  b_list = []

  # to save CI time
  if os.getenv('CI', False):
    n_epoch = 1

  for epoch in range(n_epoch):
    # Originally, we would feed each data point to the SGD optimizer.
    # Here we are feeding the whole batch of the data, instead. ;)
    optimizer.zero_grad()
    yhat = model(x_tensor)
    loss = criterion(yhat, y_tensor)
    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    cost.append(loss.item())

    w_param, b_param = list(model.parameters())
    w_list.append(w_param.item())
    b_list.append(b_param.item())
  # end epoch loop

  return cost, w_list, b_list



In [ ]:
%%time
cost_list, w_list, b_list = train(
    y_tensor, x_tensor,
    custom_model, optimizer, criterion
)



* How fast does the loss function decreases?<br>손실함수가 얼마 정도 빨리 감소하는가?



In [ ]:
plt.loglog(cost_list, '.')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.grid(True)



In [ ]:
y_hat_trained_tensor = custom_model(x_tensor)
plot(x_array, y_true, y_measurement, y_hat_trained_tensor, 'after training')



In [ ]:
def calc_cost_surface(
    w_range:int, b_range:int,
    X:torch.tensor, Y:torch.tensor,
    n_samples:int=31,
):
  # inspired by
  # https://www.coursera.org/learn/deep-neural-networks-with-pytorch/

  w_vec = np.linspace(0, w_range, n_samples)
  b_vec = np.linspace(0, b_range, n_samples)

  w_grid, b_grid = np.meshgrid(w_vec, b_vec)

  x = X.numpy().reshape(1, -1)
  y = Y.numpy().reshape(1, -1)

  x_one = np.vstack([
    x,
    np.ones_like(x)
  ])

  w_flat = w_grid.flatten()
  b_flat = b_grid.flatten()
  wb = np.column_stack([w_flat, b_flat])

  assert wb.shape[-1] == x_one.shape[0], (
      '\n'
      f"w_flat.shape = {w_flat.shape}\n"
      f"wb.shape = {wb.shape}\n"
      f"x_one.shape = {x_one.shape}\n"
  )
  yhat = wb @ x_one

  ones_y = np.ones((len(w_flat), 1))

  # using numpy broadcasting along the first dimension
  # yhat [pq, n]
  # y[1, n]
  error = yhat - y
  z_flat = np.mean(error**2, axis=1)

  Z = z_flat.reshape(*w_grid.shape)

  return w_grid, b_grid, Z



In [ ]:
def plot_cost_surf(X:torch.tensor, Y:torch.tensor, w_range:int=2, b_range:int=5):
  w_grid, b_grid, Z = calc_cost_surface(w_range, b_range, X, Y)
  _, ax = plt.subplots(
      1, 1,
      subplot_kw={'projection':'3d'},
      figsize=(16, 9),
  )
  ax.plot_surface(w_grid, b_grid, Z, alpha=0.5)
  ax.set_xlabel('w')
  ax.set_ylabel('b')
  ax.set_zlabel('l(w,b)')
  ax.grid(True)

  return ax



In [ ]:
ax = plot_cost_surf(x_tensor, y_tensor)
ax.plot(w_list, b_list, cost_list, '.');



## Final Bell<br>마지막 종



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

